# IBM Ponder This Challenge: Jan 2021

A robot is distributing COVID-19 vaccines on an NxN grid. At the beginning, the robot is located at the (0,0) cell facing UP, and every cell in the grid is marked as having received zero doses of the vaccine. The robot's goal is to ensure that each cell on the grid gets two doses of vaccine by visiting each cell twice.

Unfortunately, the robot was attacked by anti-vaccine bots, causing its navigation system to malfunction. As a result, the robot now acts like this at every step:

If the cell was vaccinated 0 times, it administers one dose (raising the cell's status from "0" to "1") and turns 90 degrees CLOCKWISE.
If the cell was vaccinated 1 time, it administers one dose (raising the cell's status from "1" to "2") and turns 90 degrees COUNTERCLOCKWISE.
If the cell was vaccinated 2 times, the robot does not change direction.
The robot then takes one step in the direction it is currently facing. If the step takes it out of the grid, it returns from the other side (i.e., coordinate arithmetic is modulo N).
Here is an example of how the robot acts on a 4x4 grid for the first few steps (the cell (0,0) is pictured at the top-left corner). The current cell is marked by square brackets; recall that the robot begins by facing upwards.
```
[0]  0   0   0
 0   0   0   0
 0   0   0   0
 0   0   0   0
            
 1  [0]  0   0
 0   0   0   0
 0   0   0   0
 0   0   0   0
            
 1   1   0   0
 0  [0]  0   0
 0   0   0   0
 0   0   0   0
    
 1   1   0   0
[0]  1   0   0
 0   0   0   0
 0   0   0   0
        
[1]  1   0   0
 1   1   0   0
 0   0   0   0
 0   0   0   0
        
 2   1   0  [0]
 1   1   0   0
 0   0   0   0
 0   0   0   0
 
```
It can be seen already that for N=4, the robot cannot administer two doses to the whole grid. Luckily, some of the anti-vaccine bots remain and can be used to our benefit. We can change some cells of the grid to contain the value "B" and give the robot an additional rule:

If the cell's status is "B", the robot avoids the anti-vaccine bots, administers no vaccine, and turns 90 degrees COUNTERCLOCKWISE.
The robot then takes one step as usual (the anti-vaccine bot cannot harm it).
For example:
```

[0]  B   0   0
 0   0   0   0
 0   0   0   0
 0   0   0   0
            
 1  [B]  0   0
 0   0   0   0
 0   0   0   0
 0   0   0   0
            
 1   B   0   0
 0   0   0   0
 0   0   0   0
 0  [0]  0   0
```

We do not need the "B" cells to be vaccinated.

Your goal is to find a placement of two "B" cells that ensures the rest of the grid will reach "2" status eventually for N=50. Give your answer in the following format: [(a,b), (x,y)] where (a,b) are the coordinates of the first "B" cell and (x,y) are the coordinates of the second "B" cell.

In [61]:
import numpy as np

#this function will 
# return a numpy array filled with zeros and -1 where the B bots are
def DebugPrint(myString):
    if DebugMode:
        print(myString)
def InitGrid(N, B_indexes_l):
    numBs = len(B_indexes_l)
    Grid = np.zeros((N,N))

    for indexes in B_indexes_l: 
        Grid[indexes[0],indexes[1]] = -1

    return Grid
def RotateRobot(Cardinality, direction_s):
    if direction_s == 'clockwise':
        if Cardinality == 3:
            Cardinality = 0
        else:
            Cardinality += 1

    elif direction_s == 'counterclockwise':
        if Cardinality == 0: 
            Cardinality = 3
        else:
            Cardinality -= 1

    elif direction_s == 'none':
        Cardinality += 0

    else: 
        print("wrong input")
    
    return Cardinality

def MoveRobotForward(Robot_pos,Cardinality):
    #robot pos = [x,y] = [row,col]
    x = Robot_pos[0]
    y = Robot_pos[1]

    if Cardinality == 0:
        #move up 
        if x == 0: 
            #gotta wrap around
            x = N - 1
        else:
            x -= 1
    elif Cardinality == 1: 
        #move to the right
        if y == N-1:
            #wrap around
            y = 0
        else: 
            y +=1
    elif Cardinality == 2: 
        #move down
        if x == N-1:
            #wrap around
            x = 0
        else: 
            x += 1
    elif Cardinality == 3: 
        #move left
        if y == 0: 
            #wrap around
            y = N - 1
        else: 
            y -= 1
    
    return [x,y]


def RunSimulation(Grid,Current_rob_pos):
    #Sanity checks
    if all(i >= N for i in Current_rob_pos[0:1]) or all(i < 0 for i in Current_rob_pos[0:1]) or Current_rob_pos[2] >= 4 or Current_rob_pos[2] < 0 : 
        print("invalid rob position")
        print("Current Rob Positino {}".format(Current_rob_pos))
        return

    GridVal = Grid[Current_rob_pos[0],Current_rob_pos[1]]

    if GridVal == 0: 
        #vaccinate 
        newGridVal = 1
        #rotate clockwise
        newCard = RotateRobot(Current_rob_pos[2],'clockwise')
        #move forward
        newPost = MoveRobotForward(Current_rob_pos[0:2],newCard)
        
    elif GridVal == 1: 
        #vaccinate 
        newGridVal = 2
        #rotate counter clockwise
        newCard = RotateRobot(Current_rob_pos[2],'counterclockwise')
        #move forward
        newPost = MoveRobotForward(Current_rob_pos[0:2],newCard)

    elif GridVal == 2: 
        #keep vaccinated
        newGridVal = 2
        #rotate none
        newCard = RotateRobot(Current_rob_pos[2],'none')
        #move forward
        newPost = MoveRobotForward(Current_rob_pos[0:2],newCard)

    elif GridVal == -1: 
        #keep unvaccinated
        newGridVal = -1
        #rotate counter clockwise
        newCard = RotateRobot(Current_rob_pos[2],'none')
        #move forward
        newPost = MoveRobotForward(Current_rob_pos[0:2],newCard)

    else:
        print("Unidentified value in grid")
        print("Grid value: {}".format(GridVal))
        print("Grid: ")
        print(Grid)
        return
        
    #upate positions and grid value
    New_rob_pos = [newPost[0],newPost[1], newCard]
    Grid[Current_rob_pos[0],Current_rob_pos[1]] = newGridVal

    return Grid, New_rob_pos

def gridChanging(oldGrid,Grid):
    val = np.sum(np.abs(Grid.flatten()) - np.abs(oldGrid.flatten()))
    DebugPrint(val)
    return val > 0

def EveryoneVaccinated(Grid,B_indexes_l):
    tempCopy = Grid.copy()
    for indexes in B_indexes_l:
        tempCopy[indexes[0],indexes[1]] = 2
    val = np.sum(tempCopy.flatten())
    DebugPrint("val: {}".format(val))
    if  val == N*N*2:
        return True
    else: 
        return False


    




# Brute force Solution
## We'll first attempt to solve for the correct position by running through all the possbile placements of B. This requires an enumeration of all possible end positions. For a Grid of size NxN that is the N^2 triangle number = N^2 + 1 choose 2


In [91]:
N = 6
DebugMode = False
Current_rob_pos = [0,0,0] 
BList = [[[i,j]] for i in range(N) for j in range(N)]

for B_indexes_l in BList: 
    DebugPrint(B_indexes_l)
    ChangeCount = 0
    Grid = InitGrid(N,B_indexes_l)

    while ChangeCount < N+1:
    #for step in range(MaxSteps):
        if EveryoneVaccinated(Grid,B_indexes_l):
            break

        oldGrid = Grid.copy()
        Grid , Current_rob_pos  =  RunSimulation(Grid,Current_rob_pos)

        if not gridChanging(oldGrid,Grid):
            ChangeCount += 1  
            DebugPrint('ChangeCount: {}'.format(ChangeCount))
        else:
            ChangeCount = 0
        DebugPrint(Grid)
        DebugPrint(Current_rob_pos)
    print("Simulation is done!")

    if EveryoneVaccinated(Grid,B_indexes_l):
        print("Everyone was vaccinated :)\n")
        SolutionFound = True
        break
    else: 
        print("Everyone is gonna die :(\n")
        SolutionFound = False
if SolutionFound:
    print("Final Grid:")
    print(Grid)
    print("Valid B position: {}".format(B_indexes_l))

Simulation is done!
Everyone is gonna die :(

Simulation is done!
Everyone is gonna die :(

Simulation is done!
Everyone is gonna die :(

Simulation is done!
Everyone is gonna die :(

Simulation is done!
Everyone is gonna die :(

Simulation is done!
Everyone is gonna die :(

Simulation is done!
Everyone is gonna die :(

Simulation is done!
Everyone is gonna die :(

Simulation is done!
Everyone is gonna die :(

Simulation is done!
Everyone is gonna die :(

Simulation is done!
Everyone is gonna die :(

Simulation is done!
Everyone is gonna die :(

Simulation is done!
Everyone is gonna die :(

Simulation is done!
Everyone is gonna die :(

Simulation is done!
Everyone is gonna die :(

Simulation is done!
Everyone is gonna die :(

Simulation is done!
Everyone is gonna die :(

Simulation is done!
Everyone is gonna die :(

Simulation is done!
Everyone is gonna die :(

Simulation is done!
Everyone is gonna die :(

Simulation is done!
Everyone is gonna die :(

Simulation is done!
Everyone is go

In [97]:
N = 2
BList = [[[i,j],[a,b]] for i in range(N) for j in range(N) for a in range(N) for b in range(N) if [i,j] != [a,b]]
print(BList)
print(len(BList))

def func(listOfItems):
    seen = set()  #use set to keep track of already seen items, sets provide O(1) lookup  
    for x in listOfItems:
        for y in listOfItems:
            if x!=y and (y,x) not in seen:
                seen.add((x,y)) 
                yield x,y

lis = [(i,j) for i in range(N) for j in range(N)]
list(func(lis))

[[[0, 0], [0, 1]], [[0, 0], [1, 0]], [[0, 0], [1, 1]], [[0, 1], [0, 0]], [[0, 1], [1, 0]], [[0, 1], [1, 1]], [[1, 0], [0, 0]], [[1, 0], [0, 1]], [[1, 0], [1, 1]], [[1, 1], [0, 0]], [[1, 1], [0, 1]], [[1, 1], [1, 0]]]
12


[((0, 0), (0, 1)),
 ((0, 0), (1, 0)),
 ((0, 0), (1, 1)),
 ((0, 1), (1, 0)),
 ((0, 1), (1, 1)),
 ((1, 0), (1, 1))]